<a href="https://colab.research.google.com/github/YeChen/RevealAPITest/blob/main/reveal_api_test_11_field_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
from google.colab import userdata
from IPython.display import display, HTML

# Step 1: Retrieve USER and PASSWORD secrets from Colab
username = userdata.get("DEMO_USER")
password = userdata.get("DEMO_PASSWORD")

if not username or not password:
    raise ValueError("Secrets USER and PASSWORD must be set in Colab.")

def print_color(message, color):
    display(HTML(f'<p style="color: {color};">{message}</p>'))

# Step 2: Define the authentication function
def authenticate(username, password):
    """
    Authenticates with the Reveal API using username and password.
    Returns loginSessionId and authenticatedUserId if authentication is successful.
    """
    login_url = "https://consulting.us-east-1.reveal11.cloud/rest/api/v2/login"
    login_data = {
        "username": username,
        "password": password
    }

    headers = {
        "Content-Type": "application/json",
        "User-Agent": "Reveal-API-Tester/1.0"
    }

    response = requests.post(login_url, json=login_data, headers=headers)
    response.raise_for_status()  # Raise an error if the request fails

    login_response = response.json()
    login_session_id = login_response.get("loginSessionId")
    authenticated_user_id = login_response.get("userId")

    if not login_session_id or not authenticated_user_id:
        raise ValueError("Failed to retrieve loginSessionId and authenticatedUserId from login response.")

    print("Authentication successful!")
    return login_session_id, authenticated_user_id

# Step 3: Define the function to retrieve projects
def get_projects(login_session_id, authenticated_user_id):
    """
    Retrieves a list of projects for the authenticated user and returns the first project's projectId.
    """
    projects_url = "https://consulting.us-east-1.reveal11.cloud/rest/api/v2/projects"

    headers = {
        "Authorization": f"Bearer {login_session_id}",
        "incontrolauthtoken": login_session_id,
        "User-Agent": "Reveal-API-Tester/1.0",
        "Accept": "application/json"
    }

    params = {"userId": authenticated_user_id}
    response = requests.get(projects_url, headers=headers, params=params)
    response.raise_for_status()

    projects_response = response.json()
    projects = projects_response.get("results", [])

    if not projects:
        print("No projects found.")
        return None

    # Extract the projectId of the first project
    first_project = projects[0]
    project_id = first_project.get("projectId")
    project_name = first_project.get("projectName", "Unnamed Project")
    print(f"First project found: {project_name} (projectId: {project_id})")
    return project_id

# Step 4: Define the function to get fields for a project
def get_project_fields(login_session_id, project_id):
    """
    Retrieves a list of fields for the specified projectId and prints fieldName, displayName, and dataTypeDsc.
    """
    fields_url = f"https://consulting.us-east-1.reveal11.cloud/rest/api/v2/{project_id}/fields"

    headers = {
        "Authorization": f"Bearer {login_session_id}",
        "incontrolauthtoken": login_session_id,
        "User-Agent": "Reveal-API-Tester/1.0",
        "Accept": "application/json"
    }

    response = requests.get(fields_url, headers=headers)
    response.raise_for_status()

    fields_response = response.json()
    fields = fields_response.get("results", [])

    if fields:
        print("Fields in the project:")
        for field in fields:
            field_name = field.get('fieldName', 'Unknown')
            display_name = field.get('displayName', 'Unknown')
            data_type_dsc = field.get('dataTypeDesc', 'Unknown')
            print(f"- fieldName: {field_name}, displayName: {display_name}, dataTypeDsc: {data_type_dsc}")
    else:
        print("No fields found for the project.")

# Main Execution
try:
    # Step 1: Authenticate to get session ID and authenticated user ID
    session_id, authenticated_user_id = authenticate(username, password)

    # Step 2: Retrieve the first project ID
    project_id = get_projects(session_id, authenticated_user_id)

    # Step 3: If project_id is found, retrieve and print fields
    if project_id:
        get_project_fields(session_id, project_id)

except requests.exceptions.RequestException as e:
    print_color(f"API request failed: {e}", "red")
except ValueError as e:
    print_color(f"Error: {e}", "red")


Authentication successful!
First project found: DanProject2 (projectId: 1795)
Fields in the project:
- fieldName: BATCHID, displayName: Batch ID, dataTypeDsc: Value used to group a set of documents.
- fieldName: ITEMID, displayName: Item ID, dataTypeDsc: The Reveal unique identifier.
- fieldName: PARENT_ITEMID, displayName: Parent ID, dataTypeDsc: The Reveal family identifier. Populated with the ITEMID of the parent record.
- fieldName: BEGDOC, displayName: Begin Number, dataTypeDsc: Beginning review control number.
- fieldName: ENDDOC, displayName: End Number, dataTypeDsc: Ending review control number.
- fieldName: MD5_HASH, displayName: Duplicate ID, dataTypeDsc: Hash value used to identify duplicates
- fieldName: CUSTODIAN_NAME, displayName: Custodian, dataTypeDsc: Custodian name used to group documents.
- fieldName: EXPORTED_NATIVE_FILE, displayName: Exported Native File, dataTypeDsc: Location of native file within processing system.
- fieldName: RELATIVE_PATH, displayName: Folder,